In [16]:
!pip install facenet-pytorch
!pip install Pillow

In [17]:
!pip install opencv-python

In [18]:
pip install facenet-pytorch torchvision numpy matplotlib pillow

Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
import torch
import numpy as np
import cv2
from PIL import Image
from torchvision import transforms
from facenet_pytorch import InceptionResnetV1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor()
])

known_embeddings = []
known_names = []
base_folder = "IMAGES"

for person_name in os.listdir(base_folder):
    person_folder = os.path.join(base_folder, person_name)
    if not os.path.isdir(person_folder):
        continue

    for img_name in os.listdir(person_folder):
        img_path = os.path.join(person_folder, img_name)

        try:
            image = Image.open(img_path).convert("RGB")
            img_tensor = transform(image).unsqueeze(0).to(device)

            with torch.no_grad():
                embedding = model(img_tensor).squeeze().cpu().numpy()

            known_embeddings.append(embedding)
            known_names.append(person_name)
        except Exception as e:
            print(f"⚠️ Failed to load {img_path}: {e}")

print(f"✅ Loaded {len(known_embeddings)} face embeddings.")

print("📷 Starting webcam... Press 'q' to quit")
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Couldn't access webcam frame.")
        break

    image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        test_embedding = model(img_tensor).squeeze().cpu().numpy()

    similarities = [
        np.dot(test_embedding, emb) / (np.linalg.norm(test_embedding) * np.linalg.norm(emb))
        for emb in known_embeddings
    ]

    if similarities:
        best_match_index = np.argmax(similarities)
        best_score = similarities[best_match_index]
    else:
        best_score = 0

    if best_score > 0.6:
        name = known_names[best_match_index]
        text = f"✅ {name} ({best_score:.2f})"
        color = (0, 255, 0)
    else:
        text = "❌ Unknown"
        color = (0, 0, 255)

    cv2.putText(frame, text, (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


✅ Loaded 2 face embeddings.
📷 Starting webcam... Press 'q' to quit
